# Covertype Data Set Preprocessing

In [2]:
%pip install numpy pandas scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [3]:
import csv
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

### Set the paths

In [28]:
ORIGINAL_DATASET_ROOT="original_data"
PROCESSED_DATASET_ROOT="processed_data"

ORIGINAL_DATASET_PATH = f'{ORIGINAL_DATASET_ROOT}/covtype.data'

FULL_DATASET = f'{PROCESSED_DATASET_ROOT}/covertype.csv'
SMALL_DATASET= f'{PROCESSED_DATASET_ROOT}/covertype_small.csv'
TRAINING_DATASET=f'{PROCESSED_DATASET_ROOT}/covertype_training.csv'
TRAINING_DATASET_WITH_MISSING = f'{PROCESSED_DATASET_ROOT}/covertype_training_missing.csv'
EVALUATION_DATASET=f'{PROCESSED_DATASET_ROOT}/covertype_evaluation.csv'
EVALUATION_DATASET_WITH_ANOMALIES=f'{PROCESSED_DATASET_ROOT}/covertype_evaluation_anomalies.csv'
SERVING_DATASET=f'{PROCESSED_DATASET_ROOT}/covertype_serving.csv'


### Download the original dataset

In [29]:
%%bash -s $ORIGINAL_DATASET_ROOT $PROCESSED_DATASET_ROOT
mkdir -p $1
mkdir -p $2
cd original_data
wget http://archive.ics.uci.edu/static/public/31/covertype.zip -O covertype.zip
unzip -o covertype.zip
rm covertype.zip
gunzip -f covtype.data.gz

--2023-10-11 22:38:37--  http://archive.ics.uci.edu/static/public/31/covertype.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252


Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘covertype.zip’

     0K .......... .......... .......... .......... ..........  502K
    50K .......... .......... .......... .......... .......... 1.00M
   100K .......... .......... .......... .......... .......... 5.02M
   150K .......... .......... .......... .......... .......... 1.27M
   200K .......... .......... .......... .......... .......... 5.14M
   250K .......... .......... .......... .......... .......... 5.34M
   300K .......... .......... .......... .......... .......... 5.05M
   350K .......... .......... .......... .......... .......... 2.42M
   400K .......... .......... .......... .......... .......... 4.89M
   450K .......... .......... .......... .......... .......... 5.09M
   500K .......... .......... .......... .......... .......... 5.24M
   550K .......... .......... .......... .......... .....

Archive:  covertype.zip
  inflating: covtype.data.gz         
  inflating: covtype.info            
  inflating: old_covtype.info        


## Preprocess the original dataset

### Load the dataset

In [30]:
df = pd.read_csv(ORIGINAL_DATASET_PATH, header=None)
print(df.shape)
df.head()

(581012, 55)


,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


### Configure soil type and wilderness area domains

In [31]:
soil_type = [
"1", "C2702", "Cathedral family - Rock outcrop complex, extremely stony.",
"2", "C2703", "Vanet - Ratake families complex, very stony.",
"3", "C2704", "Haploborolis - Rock outcrop complex, rubbly.",
"4", "C2705", "Ratake family - Rock outcrop complex, rubbly.",
"5", "C2706", "Vanet family - Rock outcrop complex complex, rubbly.",
"6", "C2717", "Vanet - Wetmore families - Rock outcrop complex, stony.",
"7", "C3501", "Gothic family.",
"8", "C3502", "Supervisor - Limber families complex.",
"9", "C4201", "Troutville family, very stony.",
"10", "C4703", "Bullwark - Catamount families - Rock outcrop complex, rubbly.",
"11", "C4704", "Bullwark - Catamount families - Rock land complex, rubbly.",
"12", "C4744", "Legault family - Rock land complex, stony.",
"13", "C4758", "Catamount family - Rock land - Bullwark family complex, rubbly.",
"14", "C5101", "Pachic Argiborolis - Aquolis complex.",
"15", "C5151", "unspecified in the USFS Soil and ELU Survey.",
"16", "C6101", "Cryaquolis - Cryoborolis complex.",
"17", "C6102", "Gateview family - Cryaquolis complex.",
"18", "C6731", "Rogert family, very stony.",
"19", "C7101", "Typic Cryaquolis - Borohemists complex.",
"20", "C7102", "Typic Cryaquepts - Typic Cryaquolls complex.",
"21", "C7103", "Typic Cryaquolls - Leighcan family, till substratum complex.",
"22", "C7201", "Leighcan family, till substratum, extremely bouldery.",
"23", "C7202", "Leighcan family, till substratum - Typic Cryaquolls complex.",
"24", "C7700", "Leighcan family, extremely stony.",
"25", "C7701", "Leighcan family, warm, extremely stony.",
"26", "C7702", "Granile - Catamount families complex, very stony.",
"27", "C7709", "Leighcan family, warm - Rock outcrop complex, extremely stony.",
"28", "C7710", "Leighcan family - Rock outcrop complex, extremely stony.",
"29", "C7745", "Como - Legault families complex, extremely stony.",
"30", "C7746", "Como family - Rock land - Legault family complex, extremely stony.",
"31", "C7755", "Leighcan - Catamount families complex, extremely stony.",
"32", "C7756", "Catamount family - Rock outcrop - Leighcan family complex, extremely stony.",
"33", "C7757", "Leighcan - Catamount families - Rock outcrop complex, extremely stony.",
"34", "C7790", "Cryorthents - Rock land complex, extremely stony.",
"35", "C8703", "Cryumbrepts - Rock outcrop - Cryaquepts complex.",
"36", "C8707", "Bross family - Rock land - Cryumbrepts complex, extremely stony.",
"37", "C8708", "Rock outcrop - Cryumbrepts - Cryorthents complex, extremely stony.",
"38", "C8771", "Leighcan - Moran families - Cryaquolls complex, extremely stony.",
"39", "C8772", "Moran family - Cryorthents - Leighcan family complex, extremely stony.",
"40", "C8776", "Moran family - Cryorthents - Rock land complex, extremely stony.",
]

wilderness_area = [
"Rawah", "Rawah Wilderness Area",
"Neota", "Neota Wilderness Area",
"Commanche", "Comanche Peak Wilderness Area",
"Cache", "Cache la Poudre Wilderness Area"
]

### Map one-hot encoded values to categorical domains

In [32]:
soil = df.loc[:, 14:53].apply(lambda x: soil_type[1::3][x.to_numpy().nonzero()[0][0]], axis=1)
soil

0         C7745
1         C7745
2         C4744
3         C7746
4         C7745
          ...  
581007    C2703
581008    C2703
581009    C2703
581010    C2703
581011    C2703
Length: 581012, dtype: object

In [20]:
wilderness = df.loc[:, 10:13].apply(lambda x: wilderness_area[0::2][x.to_numpy().nonzero()[0][0]], axis=1)
wilderness

0             Rawah
1             Rawah
2             Rawah
3             Rawah
4             Rawah
            ...    
581007    Commanche
581008    Commanche
581009    Commanche
581010    Commanche
581011    Commanche
Length: 581012, dtype: object

### Create a dataset with column names and categorical values replacing one-hot encoded soil type and wilderness areas

In [33]:
COLUMN_NAMES = [
    'Elevation', 
    'Aspect', 
    'Slope', 
    'Horizontal_Distance_To_Hydrology',
    'Vertical_Distance_To_Hydrology',
    'Horizontal_Distance_To_Roadways',
    'Hillshade_9am',
    'Hillshade_Noon',
    'Hillshade_3pm',
    'Horizontal_Distance_To_Fire_Points',
    'Wilderness_Area',
    'Soil_Type',
    'Cover_Type']

df_full = pd.concat([df.loc[:, 0:9], wilderness, soil, df.loc[:, 54]], axis=1, ignore_index=True)
df_full.columns = COLUMN_NAMES
df_full

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,Rawah,C7745,5
1,2590,56,2,212,-6,390,220,235,151,6225,Rawah,C7745,5
2,2804,139,9,268,65,3180,234,238,135,6121,Rawah,C4744,2
3,2785,155,18,242,118,3090,238,238,122,6211,Rawah,C7746,2
4,2595,45,2,153,-1,391,220,234,150,6172,Rawah,C7745,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,Commanche,C2703,3
581008,2391,152,19,67,12,95,240,237,119,845,Commanche,C2703,3
581009,2386,159,17,60,7,90,236,241,130,854,Commanche,C2703,3
581010,2384,170,15,60,5,90,230,245,143,864,Commanche,C2703,3


### Convert the label to 0-6 range

In [34]:
df_full['Cover_Type'] = df_full['Cover_Type'] - 1

### Save the dataset to CSV file

In [35]:
df_full.to_csv(FULL_DATASET, header=True, index=False)

In [36]:
!head $FULL_DATASET

Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
2596,51,3,258,0,510,221,232,148,6279,Rawah,C7745,4
2590,56,2,212,-6,390,220,235,151,6225,Rawah,C7745,4
2804,139,9,268,65,3180,234,238,135,6121,Rawah,C4744,1
2785,155,18,242,118,3090,238,238,122,6211,Rawah,C7746,1
2595,45,2,153,-1,391,220,234,150,6172,Rawah,C7745,4
2579,132,6,300,-15,67,230,237,140,6031,Rawah,C7745,1
2606,45,7,270,5,633,222,225,138,6256,Rawah,C7745,4
2605,49,4,234,7,573,222,230,144,6228,Rawah,C7745,4
2617,45,9,240,56,666,223,221,133,6244,Rawah,C7745,4


## Create training, validation, testing and serving splits.

In [37]:
df_full = df = pd.read_csv(FULL_DATASET, dtype={'Soil_Type': object})
df_full

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,Rawah,C7745,4
1,2590,56,2,212,-6,390,220,235,151,6225,Rawah,C7745,4
2,2804,139,9,268,65,3180,234,238,135,6121,Rawah,C4744,1
3,2785,155,18,242,118,3090,238,238,122,6211,Rawah,C7746,1
4,2595,45,2,153,-1,391,220,234,150,6172,Rawah,C7745,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,Commanche,C2703,2
581008,2391,152,19,67,12,95,240,237,119,845,Commanche,C2703,2
581009,2386,159,17,60,7,90,236,241,130,854,Commanche,C2703,2
581010,2384,170,15,60,5,90,230,245,143,864,Commanche,C2703,2


In [38]:
df_full.Soil_Type.value_counts()

Soil_Type
C7745    115247
C7202     57752
C7756     52519
C7757     45154
C7201     33373
C4703     32634
C7746     30170
C4744     29971
C7755     25666
C7700     21278
C4758     17431
C8771     15573
C8772     13806
C4704     12410
C2705     12396
C7102      9259
C8776      8750
C2703      7525
C2717      6575
C2704      4823
C7101      4021
C6102      3422
C2702      3031
C6101      2845
C7702      2589
C6731      1899
C8703      1891
C7790      1611
C2706      1597
C4201      1147
C7709      1086
C7710       946
C7103       838
C5101       599
C7701       474
C8708       298
C3502       179
C8707       119
C3501       105
C5151         3
Name: count, dtype: int64

In [39]:
df_5151 = df_full[df_full['Soil_Type']=='C5151']
df_no_5151 = df_full[df_full['Soil_Type']!='C5151']

In [40]:
df_5151

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
241543,2078,34,10,0,0,212,219,218,134,484,Cache,C5151,5
241544,2080,13,19,30,0,192,198,197,132,499,Cache,C5151,5
241545,2076,27,24,30,5,175,201,180,105,516,Cache,C5151,5


In [41]:
df_no_5151

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,Rawah,C7745,4
1,2590,56,2,212,-6,390,220,235,151,6225,Rawah,C7745,4
2,2804,139,9,268,65,3180,234,238,135,6121,Rawah,C4744,1
3,2785,155,18,242,118,3090,238,238,122,6211,Rawah,C7746,1
4,2595,45,2,153,-1,391,220,234,150,6172,Rawah,C7745,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2396,153,20,85,17,108,240,237,118,837,Commanche,C2703,2
581008,2391,152,19,67,12,95,240,237,119,845,Commanche,C2703,2
581009,2386,159,17,60,7,90,236,241,130,854,Commanche,C2703,2
581010,2384,170,15,60,5,90,230,245,143,864,Commanche,C2703,2


In [42]:
df_small, df_other = train_test_split(df_no_5151, train_size=100000, stratify=df_no_5151.Cover_Type)

In [43]:
df_train, df_other = train_test_split(df_no_5151, train_size=431009, stratify=df_no_5151.Cover_Type)
df_evaluate, df_serving = train_test_split(df_other, train_size=75000, stratify=df_other.Cover_Type)
df_serving = df_serving.drop(columns=['Cover_Type'])
print(df_train.shape)
print(df_evaluate.shape)
print(df_serving.shape)

(431009, 13)
(75000, 13)
(75000, 12)


Add some missing values to the training split.

In [44]:
df_train_missing = df_train.reset_index(drop=True)
df_train_missing.loc[0:8999, 'Horizontal_Distance_To_Hydrology'] = None
df_train_missing

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,3394,90,10,NaN,184,760,235,224,120,67,Neota,C8771,6
1,3197,100,18,NaN,92,960,246,213,89,1984,Rawah,C7745,1
2,2558,47,18,NaN,115,1048,223,198,103,1116,Commanche,C4703,1
3,3004,152,14,NaN,85,5761,236,240,131,3861,Rawah,C7102,1
4,2955,39,25,NaN,62,1986,211,175,86,1597,Commanche,C7757,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
431004,3158,142,14,808.0,222,1057,240,236,121,2106,Commanche,C7757,1
431005,2631,18,15,42.0,5,2100,207,206,133,924,Commanche,C4703,1
431006,2596,153,15,240.0,37,1528,237,240,130,2337,Commanche,C2705,2
431007,2962,317,12,60.0,7,4583,188,230,181,5444,Rawah,C7102,1


Create the evaluation split where some values of Slope are more than 90 degrees and 3 examples have 5151 code for soil type, which is not present in the training split.

In [45]:
df_evaluate_anomalies = df_evaluate.reset_index(drop=True)
df_evaluate_anomalies.loc[0:4, 'Slope'] = 110
df_evaluate_anomalies = pd.concat([df_evaluate_anomalies, df_5151])
df_evaluate_anomalies

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area,Soil_Type,Cover_Type
0,2935,41,110,228,40,693,221,221,134,1710,Commanche,C7756,0
1,3069,269,110,90,32,2333,185,244,199,531,Commanche,C7756,1
2,2987,152,110,60,8,3331,241,226,94,993,Commanche,C4758,1
3,2846,341,110,67,4,2455,197,224,165,612,Commanche,C7755,1
4,3160,13,110,108,25,604,204,208,138,2010,Neota,C7202,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
74998,2993,303,20,210,47,604,160,226,204,1556,Commanche,C7700,1
74999,3072,58,3,30,0,1819,222,233,147,1913,Rawah,C7202,0
241543,2078,34,10,0,0,212,219,218,134,484,Cache,C5151,5
241544,2080,13,19,30,0,192,198,197,132,499,Cache,C5151,5


In [46]:
df_evaluate_anomalies.Soil_Type.value_counts()

Soil_Type
C7745    14743
C7202     7427
C7756     6889
C7757     5756
C7201     4264
C4703     4135
C4744     3901
C7746     3852
C7755     3201
C7700     2844
C4758     2354
C8771     2052
C8772     1742
C4704     1618
C2705     1600
C7102     1232
C8776     1089
C2703      983
C2717      836
C2704      646
C7101      524
C6102      450
C2702      406
C6101      374
C7702      362
C6731      256
C8703      235
C7790      227
C2706      196
C4201      173
C7709      145
C7710      121
C7103      115
C5101       99
C7701       66
C8708       36
C3502       23
C3501       16
C8707       12
C5151        3
Name: count, dtype: int64

### Save the splits to local files.

In [47]:
df_train.to_csv(TRAINING_DATASET, header=True, index=False)
df_small.to_csv(SMALL_DATASET, header=True, index=False)
df_train_missing.to_csv(TRAINING_DATASET_WITH_MISSING, header=True, index=False)
df_evaluate.to_csv(EVALUATION_DATASET, header=True, index=False)
df_evaluate_anomalies.to_csv(EVALUATION_DATASET_WITH_ANOMALIES, header=True, index=False)
df_serving.to_csv(SERVING_DATASET, header=True, index=False)

### Create Bucket for files

In [53]:
BUCKET_NAME="gcp-arch-center-vmm"
REGION="us-central1"
!gsutil mb -l $REGION gs://$BUCKET_NAME 

Creating gs://gcp-arch-center-vmm/...


### Copy the splits to GCS

In [31]:
!gsutil cp $FULL_DATASET gs://$BUCKET_NAME/covertype/full/dataset.csv
!gsutil cp $SMALL_DATASET gs://$BUCKET_NAME/covertype/small/dataset.csv
!gsutil cp $TRAINING_DATASET gs://$BUCKET_NAME/covertype/training/dataset.csv
!gsutil cp $TRAINING_DATASET_WITH_MISSING gs://$BUCKET_NAME/covertype/training_missing/dataset.csv
!gsutil cp $EVALUATION_DATASET gs://$BUCKET_NAME/covertype/evaluation/dataset.csv
!gsutil cp $EVALUATION_DATASET_WITH_ANOMALIES gs://$BUCKET_NAME/covertype/evaluation_anomalies/dataset.csv
!gsutil cp $SERVING_DATASET gs://$BUCKET_NAME/covertype/serving/dataset.csv

Copying file://../covertype.csv [Content-Type=text/csv]...
- [1 files][ 30.5 MiB/ 30.5 MiB]                                                
Operation completed over 1 objects/30.5 MiB.                                     
Copying file://../covertype_small.csv [Content-Type=text/csv]...
/ [1 files][  5.3 MiB/  5.3 MiB]                                                
Operation completed over 1 objects/5.3 MiB.                                      
Copying file://../covertype_training.csv [Content-Type=text/csv]...
- [1 files][ 22.7 MiB/ 22.7 MiB]                                                
Operation completed over 1 objects/22.7 MiB.                                     
Copying file://../covertype_training_missing.csv [Content-Type=text/csv]...
- [1 files][ 23.4 MiB/ 23.4 MiB]                                                
Operation completed over 1 objects/23.4 MiB.                                     
Copying file://../covertype_evaluation.csv [Content-Type=text/csv]...
/ [1 files